In [1]:
import pandas as pd
from pymongo import MongoClient

In [2]:
host = "localhost"
port = "27017"
username = "accretioadmin"
password = "adminaccretio&2017"
authSource = "admin"

In [3]:
def _connect_mongo(db):
    if username and password:
        conn = MongoClient(host=host,
                            port=int(port),
                            username=username,
                            password=password,
                           authSource=authSource)
    else:
        conn = MongoClient(host, port)
        
    return conn[db]    


def read_mongo(db, collection, query={}):
    db = _connect_mongo(db=db)
    cursor = db[collection].find(query)
    df =  pd.DataFrame(list(cursor))
    return df

In [4]:
df = read_mongo("connectTimeline", "post")

In [5]:
df = df[["_id","registrationNumber","body"]]

In [6]:
df.head()

,_id,registrationNumber,body
0,5bae630cdb5d87000977308d,00115,"Ooredoo, Acteur essentiel du secteur des nouve..."
1,5bae6c48db5d870009773094,00115,Joyeux Advy'nniversaire pour tout le groupe !\...
2,5bae93c1a5b16a0001ef4e4c,00132,Advyteam souhaite la bienvenue à tous ses coll...
3,5bae9661a5b16a0001ef4e4e,00086,Une évolution bien marquante ! qui est d'accor...
4,5bae968aa5b16a0001ef4e51,00086,Accretio passe à la vitesse suivante 🙂


In [7]:
df = df.dropna()

# Data cleaning

In [8]:
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

In [9]:
import re
import emoji

def custom_stopwords(text):
    #Case Normalization
    text = text.lower()
    #text = text.replace("mamda","accretio")
    #cleanr = re.compile('<.*?>')
    for_html_tags = re.sub(r'<.*?>', '', text)
    for_links = re.sub("https?:\/\/.*[\r\n]*", "", for_html_tags)  
    for_alpha_numeric = re.sub(r'[^a-zA-Z0-9 àâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]+', '', for_links)
    # remove emojis
    clear_text = remove_emoji(for_alpha_numeric)

    return for_alpha_numeric

In [10]:
df["body"] = df["body"].apply(custom_stopwords)

In [11]:
df.body

0       ooredoo acteur essentiel du secteur des nouvel...
1       joyeux advynniversaire pour tout le groupe on ...
2       advyteam souhaite la bienvenue à tous ses coll...
3         une évolution bien marquante  qui est daccord  
4                   accretio passe à la vitesse suivante 
                              ...                        
1767                                      zjtkl lglrtezer
1768    chers advyteamiens nous cherchons à loccasion ...
1769                                               test 1
1770                                           test ajout
1771                                               test 1
Name: body, Length: 1087, dtype: object

In [12]:
import spacy
from spacy_lefff import LefffLemmatizer
from spacy.language import Language

@Language.factory('french_lemmatizer')
def create_french_lemmatizer(nlp, name):
    return LefffLemmatizer()

In [15]:


nlp = spacy.load('fr_core_news_md')
nlp.add_pipe('french_lemmatizer', name='lefff')
text = "Emma cherche à span strong acheter une l'éxperience startup anglaise pour 1 milliard de dollard septembre"
doc = nlp(text)
for d in doc:
    if(d.pos_ != "PROPN" and d.pos_ != "NUM" and d.pos_ != "AUX"):
        text = text.replace(d.text, d.lemma_)
        print(d.text, d.pos_, d._.lefff_lemma, d.tag_, d.lemma_)

cherche VERB chercher VERB cherche
à ADP None ADP à
span VERB None VERB span
strong PRON None PRON strong
acheter VERB acheter VERB acheter
une DET un DET un
l' DET le DET le
éxperience NOUN None NOUN éxperience
startup ADJ None ADJ startup
anglaise ADJ anglais ADJ anglais
pour ADP None ADP pour
milliard NOUN milliard NOUN milliard
de ADP un ADP de
dollard NOUN None NOUN dollard
septembre NOUN septembre NOUN septembre


In [ ]:
text

In [ ]:
import spacy
nlp = spacy.load('fr_core_news_md')

doc = nlp(u"voudrais non animaux yeux dors couvre.")
for token in doc:
    print(token, token.lemma_)

In [ ]:

def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

emojis = '(｀ヘ´) 🤗⭕🤓🤔🤘🦁⭐🆗🆖🈲🤐🤗🤖🤑🆙⏩'
print(remove_emoji(emojis))

##### it’s important to normalize the case of our words so that every word is in the same case and the computer doesn’t process the same word as 2 different tokens.

#### Removing Unicode : ASCII

#### ASCII formats emojis and other non-ASCII characters as Unicode. Essentially, Unicode is a universal character encoding standard in which each character and symbol in all languages are assigned a code. Unicode is required because it is the only encoding standard that allows us to retrieve or join data using a variety of different languages but the issue is… It’s unreadable in ASCII format.